# Baselines and Formulation


In [1]:
# Install and start postgresql-11 server
!sudo apt-get -y -qq update
!wget --quiet -O - https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo apt-key add -
!echo "deb http://apt.postgresql.org/pub/repos/apt/ `lsb_release -cs`-pgdg main" |sudo tee  /etc/apt/sources.list.d/pgdg.list
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql-11 postgresql-client-11
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `donors_choose` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS donors_choose;'

!sudo -u postgres psql -U postgres -c 'CREATE DATABASE donors_choose;'

# Environment variables for connecting to the database
%env DEMO_DATABASE_NAME=donors_choose
%env DEMO_DATABASE_HOST=localhost
%env DEMO_DATABASE_PORT=5432

# Setup a database with name `donors_choose` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS donors_choose;'

!sudo -u postgres psql -U postgres -c 'CREATE DATABASE donors_choose;'

# Environment variables for connecting to the database
%env DEMO_DATABASE_NAME=donors_choose
%env DEMO_DATABASE_HOST=localhost
%env DEMO_DATABASE_PORT=5432
%env DEMO_DATABASE_USER=postgres
%env DEMO_DATABASE_PASS=postgres

OK
deb http://apt.postgresql.org/pub/repos/apt/ focal-pgdg main
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 13.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package cron.
(Reading database ... 129496 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-136ubuntu1_amd64.deb ...
Unpacking cron (3.0pl1-136ubuntu1) ...
Selecting previously unselected package libjson-perl.
Preparing to unpack .../01-libjson-perl_4.02000-2_all.deb ...
Unpacking libjson-perl (4.02000-2) ...
Selecting previously unselected package netbase.
Preparing to unpack .../02-netbase_6.1_all.deb ...
Unpacking

In [2]:
!pip install kaggle

!mkdir /root/.kaggle
!touch /root/.kaggle/kaggle.json
api_token = {"username":"ploped123","key":"eeeeba8fc52706723e4c1bcf41ae6fd3"}

import json
import zipfile
import os
import pandas as pd
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions download -c kdd-cup-2014-predicting-excitement-at-donors-choose

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
100% 924M/926M [00:31<00:00, 28.1MB/s]
100% 926M/926M [00:31<00:00, 30.9MB/s]


In [3]:
!ls /content

kdd-cup-2014-predicting-excitement-at-donors-choose.zip  sample_data


In [4]:
!unzip /content/kdd-cup-2014-predicting-excitement-at-donors-choose.zip

Archive:  /content/kdd-cup-2014-predicting-excitement-at-donors-choose.zip
  inflating: donations.csv.zip       
  inflating: essays.csv.zip          
  inflating: outcomes.csv.zip        
  inflating: projects.csv.zip        
  inflating: resources.csv.zip       
  inflating: sampleSubmission.csv.zip  


In [5]:
!unzip /content/donations.csv.zip

Archive:  /content/donations.csv.zip
  inflating: donations.csv           


In [6]:
!unzip /content/projects.csv.zip

Archive:  /content/projects.csv.zip
  inflating: projects.csv            


In [7]:
full_donations = pd.read_csv("/content/donations.csv")
full_projects = pd.read_csv("/content/projects.csv")

In [8]:
from datetime import timedelta

## Calculating base rate on full dataset

In [9]:
a = pd.merge(full_projects, full_donations, on=['projectid'], how='left')
a['in_4_months'] = (pd.to_datetime(a['donation_timestamp']) - pd.to_datetime(a['date_posted'])) < timedelta(days=120)
values = {'donation_to_project': 0.0}
a = a.fillna(value=values)
a.loc[a['in_4_months'] == False, 'donation_to_project'] = 0.0
donation_in_4_months = a.groupby(['projectid'])['donation_to_project'].sum().reset_index(name='donation_in_4_months')
df = pd.merge(full_projects, donation_in_4_months, on=['projectid'], how='left')


In [10]:
df['if_fully_funded_after_4_months'] = df['total_price_excluding_optional_support'] <= df['donation_in_4_months']
df['if_fully_funded_after_4_months'].value_counts()
print("Baserate % projects fully funded:", df['if_fully_funded_after_4_months'].mean())

Baserate % projects fully funded: 0.5480215269433126


# Baseline 1: 10% of projects that have the largest total ask amount

In [11]:
df_select = df.sort_values("total_price_excluding_optional_support", ascending=False)["if_fully_funded_after_4_months"].to_list()[:df.shape[0]//10]
tp = 1 - sum(df_select)/len(df_select)
print(f"Among the 10% of projects with the largest total ask amount, {tp*100}% are not fully funded after 4 months.")

Among the 10% of projects with the largest total ask amount, 63.34683551928202% are not fully funded after 4 months.


# Baseline 2: 10% of projects that impact the smallest number of students per dollar requested

In [12]:
df['children_helped_per_dollar'] = df['students_reached']/df['total_price_excluding_optional_support']
df_select = df.sort_values("children_helped_per_dollar")["if_fully_funded_after_4_months"].to_list()[:df.shape[0]//10]
tp = 1 - sum(df_select)/len(df_select)
print(f"Among the 10% of projects that impact the smallest number of students per dollar requested, {tp*100}% are not fully funded after 4 months.")

Among the 10% of projects that impact the smallest number of students per dollar requested, 56.67002966465389% are not fully funded after 4 months.


# Baseline 3: 10% of projects that had the lowest donations in the first month, ties broken by smallest children/dollar impact

In [19]:
a = pd.merge(full_projects, full_donations, on=['projectid'], how='left')
a['in_4_months'] = (pd.to_datetime(a['donation_timestamp']) - pd.to_datetime(a['date_posted'])) < timedelta(days=120)
a['in_1_months'] = (pd.to_datetime(a['donation_timestamp']) - pd.to_datetime(a['date_posted'])) < timedelta(days=30)
a['donation_1mo'] = a['donation_to_project'].values
values = {'donation_to_project': 0.0}
a = a.fillna(value=values)
a.loc[a['in_4_months'] == False, 'donation_to_project'] = 0.0
a.loc[a['in_1_months'] == False, 'donation_1mo'] = 0.0
donation_in_4_months = a.groupby(['projectid'])['donation_to_project'].sum().reset_index(name='donation_in_4_months')
donation_in_1_months = a.groupby(['projectid'])['donation_1mo'].sum().reset_index(name='donation_in_1_months')
df = pd.merge(full_projects, donation_in_4_months, on=['projectid'], how='left')
df = pd.merge(df, donation_in_1_months, on=['projectid'], how='left')
df['pct_funded_1mo'] = df['donation_in_1_months'] / df['total_price_excluding_optional_support']
df['children_helped_per_dollar'] = df['students_reached']/df['total_price_excluding_optional_support']

df['if_fully_funded_after_4_months'] = df['total_price_excluding_optional_support'] <= df['donation_in_4_months']
df['if_fully_funded_after_4_months'].value_counts()

df_select = df.sort_values(["pct_funded_1mo", "children_helped_per_dollar"])["if_fully_funded_after_4_months"].to_list()[:df.shape[0]//10]
tp = 1 - sum(df_select)/len(df_select)
print(f"Among the 10% of projects that impact the smallest number of students per dollar requested, {tp*100}% are not fully funded after 4 months.")

Among the 10% of projects that impact the smallest number of students per dollar requested, 81.28416329112018% are not fully funded after 4 months.
